[Lesson](https://realpython.com/introduction-to-mongodb-and-python/)

Low level and easy MongoDB language

In [1]:
import pymongo

In [2]:
#Establishing a Connection
from pymongo import MongoClient
client = MongoClient()

In [3]:
#way #1
client = MongoClient('localhost', 27017)

In [4]:
#way #2
client = MongoClient('mongodb://localhost:27017')

In [5]:
# Accessing Database
#way #1
db = client.pymongo_test
#way #2
db = client['pymongo_test']

In [6]:
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
result = posts.insert_one(post_data)
print('One post: {0}'.format(result.inserted_id))

One post: 5f99aa59bdf6c2bd87170b15


In [7]:
post_1 = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
post_2 = {
    'title': 'Virtual Environments',
    'content': 'Use virtual environments, you guys',
    'author': 'Scott'
}
post_3 = {
    'title': 'Learning Python',
    'content': 'Learn Python, it is easy',
    'author': 'Bill'
}
new_result = posts.insert_many([post_1, post_2, post_3])
print('Multiple posts: {0}'.format(new_result.inserted_ids))


Multiple posts: [ObjectId('5f99aa6dbdf6c2bd87170b16'), ObjectId('5f99aa6dbdf6c2bd87170b17'), ObjectId('5f99aa6dbdf6c2bd87170b18')]


In [8]:
bills_post = posts.find_one({'author': 'Bill'})
print(bills_post)

{'_id': ObjectId('5f99aa6dbdf6c2bd87170b18'), 'title': 'Learning Python', 'content': 'Learn Python, it is easy', 'author': 'Bill'}


In [9]:
scotts_posts = posts.find({'author': 'Scott'})
print(scotts_posts)

In [ ]:
#cursor is an iter
for post in scotts_posts:
    print(post)

MongoEngine

In [10]:
from mongoengine import *
connect('mongoengine_test', host='localhost', port=27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [11]:
import datetime

class Post(Document):
    title = StringField(required=True, max_length=200)
    content = StringField(required=True)
    author = StringField(required=True, max_length=50)
    published = DateTimeField(default=datetime.datetime.now)


   * db_field: Specify a different field name
   * required: Make sure this field is set
   * default: Use the given default value if no other value is given
   * unique: Make sure no other document in the collection has the same value for this field
   * choices: Make sure the field’s value is equal to one of the values given in an array


In [12]:
post_1 = Post(
    title='Sample Post',
    content='Some engaging content',
    author='Scott'
)
post_1.save()       # This will perform an insert
print(post_1.title)
post_1.title = 'A Better Post Title'
post_1.save()       # This will perform an atomic edit on "title"
print(post_1.title)

Sample Post
A Better Post Title


Unlike SQL, there is no rollback in mongodb, but the closest is a two phase [commit](https://docs.mongodb.com/v3.4/tutorial/perform-two-phase-commits/)

MongoDB single document are always atomic but multi-document transactions are not.
   * Atomicity: if one operation fails, the previous operation within the transaction must “rollback” to the previous state (i.e. the “nothing,” in “all or nothing”).
   * Consistency: if a major failure (i.e. network, hardware) interrupts the transaction, the database must be able to recover a consistent state.


In [13]:
post_2 = Post(content='Content goes here', author='Michael')
post_2.save()

ValidationError: ValidationError (Post:None) (Field is required: ['title'])

Since MongoDB is object oriented that means we can create our own methods

In [ ]:
class Post(Document):
    title = StringField()
    published = BooleanField()

    @queryset_manager
    def live_posts(clazz, queryset):
        return queryset.filter(published=True)
